In [ ]:
import numpy as np 
import pandas as pd
import os
import time

The following code enables everyone to transform the data for William to IAMC format in a partial way. 
To run correctly the code, you need to : 
- Create the File_To_Convert folder 
- Put the file to convert inside. 
- Install the numpy and pandas dependencies if the running of the cell above is not working. 
 - Run the following command line pip3 install pandas and pip3 install numpy 
- Put the scenario name as input after running the third cell. 
- Create a folder named Variable_Reference with two files, one called reference.xlsx which correspond to the variable used in IPCC work and another called Variable_name_IAMC.xlsx which corresponds to the translation  of William's name to IAMC's format. It is still under review and needs to be thought again. Only certain variable are translated to IAMC's format. 
 You can download the files on the following link : 

### Treat all the variables for csv and excel file 


In [ ]:

time0= time.time()

def get_last_added_file(folder_path):
    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    
    # Filter Excel files
    excel_files = [file for file in files if file.endswith('.xlsx') or file.endswith('.xls') or file.endswith('.csv')]
    
    if not excel_files:
        return None  # No Excel files found
    print()
    # Sort files by modification time (newest first)
    excel_files.sort(key=lambda x: os.path.getctime(os.path.join(folder_path, x)), reverse=True)
    
    # Return the path to the last added Excel file
    return os.path.join(folder_path, excel_files[0])





def create_folder_if_not_exists(folder_name):
    # Get current folder 
    folder_path = os.getcwd()
    
    try:
        os.makedirs("pip")
        print("DONE")
    except FileExistsError:
        
        pass
    if not os.path.exists(os.path.join(folder_path,folder_name)):
        os.makedirs(folder_name)
        print("Folder '{}' created.".format(folder_name))
    else:
        print("Folder '{}' already exists.".format(folder_name))

# Folder name to create 
folder_name_to_convert = 'File_To_Convert'
folder_name_converted = 'File_Converted'

# Get current folder 
folder_path = os.getcwd()


# Create new folder 
create_folder_if_not_exists(folder_name_converted)



In [ ]:


# Get the last upload excel file
real_folder_path= os.path.join(folder_path , folder_name_to_convert)

last_file = get_last_added_file(real_folder_path)
#last_file = "/home/tristan-martin/Documents/Stage/Documentation/Data-format/Conversion-Script/File_To_Convert/Scenario 3b.csv"

if last_file:
    print("Last added file:", last_file)
else:
    print("No files found in the folder.")

# Get the correct naming for the converted file 
filename_with_extension = os.path.basename(last_file)
splited_filename_with_extension = os.path.splitext(filename_with_extension)
print(splited_filename_with_extension)
filename = splited_filename_with_extension[0] + 'converted' + splited_filename_with_extension[1]

# Possibility to create a thread to run all the code during the wait for the user answer. 
# scenario = input("Give the name of the scenario")
scenario = splited_filename_with_extension[0]

# Read the selected file 
if splited_filename_with_extension[1] == '.xls' or splited_filename_with_extension[1] == '.xlsx':
    scenario_variable_df = pd.read_excel(last_file)
elif  splited_filename_with_extension[1] == '.csv': 
    scenario_variable_df = pd.read_csv(last_file)


scenario_variable_df

In [ ]:

#Rename the column of the file 
columns = scenario_variable_df.columns.str.replace('Unnamed:', 'Subscript')
columns = columns.str.replace('.', ' ')

# Define the string you want in column names
search_string = 'Subscript'
counter=0 

for k in range(len(columns)): 
    if search_string in columns[k]: 
        columns.values[k]= search_string + ' ' + str(counter)
        counter+=1



# Create a dictionary using zip() and dictionary comprehension
my_dict = {k: v for k, v in zip(scenario_variable_df.columns[2:6], columns[2:6])}
scenario_variable_df.rename(columns=my_dict, inplace=True)
scenario_variable_df.rename(columns={"Time": "Variable", "Year": "Unit", "Subscript":"Subscript 0"}, inplace=True)

# Insert the three columns in the right place for IAMC format
scenario_variable_df.insert(0, 'Model', "WILIAM")
scenario_variable_df.insert(1, 'Scenario', scenario)
scenario_variable_df.insert(2,"Region","World")


### Read the Excel File to change the name of the variable


In [ ]:
# Read the CSV file into a pandas DataFrame
data_name_df = pd.read_excel('Variable_Reference/Variable_name_IAMC.xlsx')
# Remplacer les tirets du bas par des espaces dans la colonne
# data_name_df['WILIAM_variable'] = data_name_df['WILIAM_variable'].str.replace('_', ' ')
# Drop rows with NaN values which corresponds to values not conserv for the final upload of data in IAMC format
data_name_df.dropna(subset=['IAMC_variable'],inplace=True)
data_name_df

In [ ]:
# Create a dict with Wiliam's name as key, IAMC's name as value
IAMC_WILIAM_name_dict = data_name_df.set_index('WILIAM_variable')['IAMC_variable'].to_dict()
print(IAMC_WILIAM_name_dict)


In [ ]:
# Replace the variable name used in William to the ones used for IAMC format. 
scenario_variable_df['Variable']= scenario_variable_df['Variable'].replace(IAMC_WILIAM_name_dict)

### Create dictionary of Energy subscripts  


In [ ]:
"""energy_sources_new = [
    "Electricity",
    "Gas",
    "Heat",
    "Hydrogen",
    "TO Liquid",
    "Solid Biomass",
    "Solid Fossil"
]
energy_sources_old = [
    "TO_elec",
    "TO_gas",
    "TO_heat",
    "TO_hydrogen",
    "TO_liquid",
    "TO_solid_bio",
    "TO_solid_fossil"
]
energy_vector_dict= dict(zip(energy_sources_old, energy_sources_new))
energy_vector_dict"""

In [ ]:


energy_final_old = [
    "FE_elec",
    "FE_gas",
    "FE_heat",
    "FE_hydrogen",
    "FE_liquid",
    "FE_solid_bio",
    "FE_solid_fossil"
]


energy_final_new = [
    "Electricity",
    "Gas",
    "Heat",
    "Hydrogen",
    "Liquid",
    "Solid Biomass",
    "Solid Fossil"
]

energy_final_dict= dict(zip(energy_final_old, energy_final_new))
energy_final_dict

In [ ]:
energy_processes_old = [
    "PROTRA_CHP_gas_fuels",
    "PROTRA_CHP_gas_fuels_CCS",
    "PROTRA_CHP_geothermal_DEACTIVATED",
    "PROTRA_CHP_liquid_fuels",
    "PROTRA_CHP_liquid_fuels_CCS",
    "PROTRA_CHP_solid_fossil",
    "PROTRA_CHP_solid_fossil_CCS",
    "PROTRA_CHP_waste",
    "PROTRA_CHP_solid_bio",
    "PROTRA_CHP_solid_bio_CCS",
    "PROTRA_HP_gas_fuels",
    "PROTRA_HP_solid_bio",
    "PROTRA_HP_geothermal",
    "PROTRA_HP_liquid_fuels",
    "PROTRA_HP_solar_DEACTIVATED",
    "PROTRA_HP_solid_fossil",
    "PROTRA_HP_waste",
    "PROTRA_PP_solid_bio",
    "PROTRA_PP_solid_bio_CCS",
    "PROTRA_PP_gas_fuels",
    "PROTRA_PP_gas_fuels_CCS",
    "PROTRA_PP_geothermal",
    "PROTRA_PP_hydropower_dammed",
    "PROTRA_PP_hydropower_run_of_river",
    "PROTRA_PP_liquid_fuels",
    "PROTRA_PP_liquid_fuels_CCS",
    "PROTRA_PP_nuclear",
    "PROTRA_PP_oceanic",
    "PROTRA_PP_solar_CSP",
    "PROTRA_PP_solar_open_space_PV",
    "PROTRA_PP_solar_urban_PV",
    "PROTRA_PP_solid_fossil",
    "PROTRA_PP_solid_fossil_CCS",
    "PROTRA_PP_waste",
    "PROTRA_PP_waste_CCS",
    "PROTRA_PP_wind_offshore",
    "PROTRA_PP_wind_onshore",
]

# Fonction pour appliquer un changement aux éléments successifs similaires
def apply_change_if_similar(lst):
    new_lst = [lst[k] for k in range(len(lst))]
    for i in range(len(lst)-1):
        
        
        if new_lst[i] == new_lst[i+1][:-4]:
            new_lst[i],new_lst[i+1] = modify_element(new_lst[i],new_lst[i+1])
        new_lst[i] = remove_underscore_before_word(new_lst[i],'fuels')
        new_lst[i] = remove_underscore_before_word(new_lst[i],'of')
        new_lst[i] = remove_underscore_before_word(new_lst[i],'river')
        new_lst[i] = remove_underscore_before_word(new_lst[i],'space')
        
            
    return new_lst


def remove_underscore_before_word(string, word):
    index = string.find(word)
    if index != -1:
        string = string[:index-1] + string[index-1:index].replace("_", " ") + string[index:]
    return string


# Fonction de modification de l'élément
def modify_element(element, element1):
    # Exemple de modification, vous pouvez remplacer cela par la modification souhaitée
    element = element + '_w/o CCS'
    element1 = element1[:-3] + 'w/ ' +element1[-3:]
    return element, element1


energy_processes_transformed = apply_change_if_similar(energy_processes_old)
energy_processes_transformed

In [ ]:


# Fonction de modification de l'élément
def modify_element(element):
    # Remplacer les underscores par des espaces et mettre chaque mot en majuscule
    return "|".join([word.capitalize() for word in element.split("_")])


energy_processes_new = []
for elem in energy_processes_transformed:
    new_elem = ''
    elem = elem[7:]
    if elem[0:2] == 'PP':
        new_elem = 'Electricity Generation'
        new_elem = new_elem + '|' + modify_element(elem[3:])
    elif elem[0:2] == 'HP': 
        new_elem = 'Heat Generation'
        new_elem = new_elem + '|' + modify_element(elem[3:])
    elif elem[0:3] == 'CHP': 
        new_elem = 'Combined Electricity and Heat Generation'
        new_elem = new_elem + '|' + modify_element(elem[4:])
    energy_processes_new.append(new_elem)

    
energy_processes_dict = dict(zip(energy_processes_old, energy_processes_new))
energy_processes_dict

### Create dictionary of Age subscript

In [ ]:
# Create the subscript list of the Wiliam model 
number_list= [[0+5*k, 4+5*k] for k in range(16)]
age_subscript_list = ['c'+str(number_list[k][0])+'c'+str(number_list[k][1]) for k in range(len(number_list))]
age_subscript_list.append('cover80')
age_subscript_list


# Create the subscript list of the IAMC Compact format
new_age_subscript_list = ['Age '+str(number_list[k][0])+'-'+str(number_list[k][1]) for k in range(len(number_list))]
new_age_subscript_list.append('Age +80')
new_age_subscript_list

age_translation_dict = dict(zip(age_subscript_list, new_age_subscript_list))
age_translation_dict

### Change the naming of Subcript in William


In [ ]:
scenario_variable_df['Subscript 0'] = scenario_variable_df['Subscript 0'].replace(energy_processes_dict)
scenario_variable_df['Subscript 1'] = scenario_variable_df['Subscript 1'].replace(energy_processes_dict)
scenario_variable_df['Subscript 2'] = scenario_variable_df['Subscript 2'].replace(energy_processes_dict)
scenario_variable_df['Subscript 3'] = scenario_variable_df['Subscript 3'].replace(energy_processes_dict)

scenario_variable_df['Subscript 0'] = scenario_variable_df['Subscript 0'].replace(energy_final_dict)
scenario_variable_df['Subscript 1'] = scenario_variable_df['Subscript 1'].replace(energy_final_dict)
scenario_variable_df['Subscript 2'] = scenario_variable_df['Subscript 2'].replace(energy_final_dict)
scenario_variable_df['Subscript 3'] = scenario_variable_df['Subscript 3'].replace(energy_final_dict)

"""df['Subscript 0'] = df['Subscript 0'].replace(energy_vector_dict)
df['Subscript 1'] = df['Subscript 1'].replace(energy_vector_dict)
df['Subscript 2'] = df['Subscript 2'].replace(energy_vector_dict)
df['Subscript 3'] = df['Subscript 3'].replace(energy_vector_dict)"""

scenario_variable_df['Subscript 0'] = scenario_variable_df['Subscript 0'].replace(age_translation_dict)
scenario_variable_df['Subscript 1'] = scenario_variable_df['Subscript 1'].replace(age_translation_dict)
scenario_variable_df['Subscript 2'] = scenario_variable_df['Subscript 2'].replace(age_translation_dict)
scenario_variable_df['Subscript 3'] = scenario_variable_df['Subscript 3'].replace(age_translation_dict)

In [ ]:
# check that the change is working 
value_counts_0 = scenario_variable_df['Subscript 2'].value_counts()
print(value_counts_0[0:50])

### Order Dataframe to the required Format by adding subscript at the end of the variable


In [ ]:
# Dict used to check if the subscripts are a country
country_dict= {'EU27':1, 'UK':1,'CHINA':1,'EASOC':1,'INDIA':1,'LATAM':1,'RUSSIA':1,'USMCA':1,'LROW':1,'AUSTRIA':1,'BELGIUM':1,'BULGARIA':1,'CROATIA':1,'CYPRUS':1,'CZECH_REPUBLIC':1,'DENMARK':1,'ESTONIA':1,'FINLAND':1,'FRANCE':1,'GERMANY':1,'GREECE':1,'HUNGARY':1,'IRELAND':1,'ITALY':1,'LATVIA':1,'LITHUANIA':1,'LUXEMBOURG':1,'MALTA':1,'NETHERLANDS':1,'POLAND':1,'PORTUGAL':1,'ROMANIA':1,'SLOVAKIA':1,'SLOVENIA':1,'SPAIN':1,'SWEDEN':1}

# Aggregate the subscripts at the end of the variable 
def aggregate_variable_name(row, counter):
    for k in range(counter):
        subscript=row["Subscript "+str(k)]

        
    
        if k==0 :
            if country_dict.get(subscript)==1: 
                #Give the name of the region for that variable
                row["Region"]=subscript
                
                row["Subscript 0"]=np.nan
                continue
        
        if not pd.isnull(subscript):
            row["Variable"]=row["Variable"]+"|"+subscript
            
    return row 


# Apply the previous function on all the lines 
scenario_variable_df= scenario_variable_df.apply(aggregate_variable_name, args=(counter,),axis=1)

# Remove the subscript columns once they have been added at the end of the variable name
drop_columns_list=["Subscript " + str(k) for k in range(counter)]
scenario_variable_df.drop(columns=drop_columns_list, inplace=True)



### Country Traduction between IAMC country and Wiliam


In [ ]:
file_path = "Variable_Reference/reference.xlsx"  # Update this with the correct path

# Read the Excel file into a pandas DataFrame
country_IPCC_df = pd.read_excel(file_path,sheet_name=2)

# Now you can work with the DataFrame 'df' as needed
country_IPCC_df.head()  # Print the first few rows of the DataFrame

In [ ]:
# Put the country in list 
country_IPCC_list = country_IPCC_df["Name"].to_list()

# Get all the region used in Wiliam
country_William_list = scenario_variable_df["Region"].unique()
country_William_list

In [ ]:
# Put the country in lowercase except for the first letter (format used by IPCC for the Countries)
new_country_Wiliam_list=[]
for country in country_William_list: 
    country = country.lower()
    country = country.title()
    country = country.replace('_',' ')
    new_country_Wiliam_list.append(country)


# Check the country that are not in the IPCC's list 
for country in new_country_Wiliam_list:
    if country not in country_IPCC_list:
        print(country)

In [ ]:
# Create a dict that associated old name with the new name. 
country_Wiliam_dict = dict(zip(country_William_list, new_country_Wiliam_list))

# Change the name by hand for all the countries that appeared in the previous cell
country_Wiliam_dict['EU27'] = 'European Union (27 member countries)' # 27 countries only different with the previous versions of 
country_Wiliam_dict['SLOVAKIA'] = 'Slovak Republic'
country_Wiliam_dict['UK'] = 'United Kingdom'
country_Wiliam_dict['EASOC'] = 'East Asia and Oceania'
country_Wiliam_dict['LATAM'] = 'Latin America' # Latin American countries 
country_Wiliam_dict['USMCA'] = 'United States, Mexico and Canada' # not correponding in an according manner, there is mexico also. 
country_Wiliam_dict['LROW'] = 'Rest of the World'

In [ ]:
country_Wiliam_dict

In [ ]:
# Replace the region name by the ones used for the IAMC format 
scenario_variable_df['Region'] = scenario_variable_df['Region'].replace(country_Wiliam_dict)
scenario_variable_df

In [ ]:
def check_countries(list1, list2): 
    for elem1 in list1: 
        if elem1 not in list2: 
            print(elem1)

# Print the countries that are not used in IPCC's format 
check_countries(scenario_variable_df['Region'].unique(), country_IPCC_list)

### Correct Mistakes in the file (Not used now)

In [ ]:
"""# Select one specific country 
scenario_variable_df = scenario_variable_df[scenario_variable_df['Region'] == 'Rest of the World']

# Select one specific variable 
specific_string = 'Final Energy'
filtered_df = scenario_variable_df[scenario_variable_df['Variable'].str.startswith(specific_string)]

# Remove Demand rows
specific_string_1 = 'Demand'
filtered_df = filtered_df[~filtered_df['Variable'].str.contains(specific_string_1)]"""

In [ ]:
scenario_variable_df['Region'].value_counts()

### Check how to explain duplicates rows 

In [ ]:
# Check for duplicate rows
duplicate_rows = scenario_variable_df[scenario_variable_df.duplicated()]

# Display duplicate rows
print("Duplicate Rows:")
print(duplicate_rows)

# Count the number of duplicate rows
num_duplicate_rows = duplicate_rows.shape[0]
print("Number of Duplicate Rows:", num_duplicate_rows)


### Writing File

In [ ]:
#Write the new excel file in the File_Converted folder
folder_file_converted=os.path.join(folder_path,folder_name_converted)

# Get the correct naming for the converted file 
filename_with_extension = os.path.basename(last_file)
filename = splited_filename_with_extension[0] + 'converted' + splited_filename_with_extension[1]

# Remove duplicate rows
scenario_variable_df.drop_duplicates(subset=['Region', 'Variable'],inplace=True)

# Write the following dataframe to excel
if splited_filename_with_extension[1] == '.xls' or splited_filename_with_extension[1] == '.xlsx':
    scenario_variable_df.to_excel(os.path.join(folder_file_converted,filename))
elif  splited_filename_with_extension[1] == '.csv': 
    scenario_variable_df.to_csv(os.path.join(folder_file_converted,filename),index=False)

print("Conversion Done")
print(time.time()-time0, "Seconds to run the code.")